In [15]:
county_level_kws = ['tax', 'loan', 'business', 'rent', 'college', 'apartments', 'office', 'insurance'] #e.g. "XX county tax"
state_level_kws = ['tax', 'loan', 'business', 'rent', 'college', 'apartments', 'office', 'insurance', 'Business plan templates', 'Small business ideas', 'How to start a small business', 'office space for rent', 'website hosting service', 'freelancing', 'side hustle', 'ecommerce'] #e.g. "tax"

In [6]:
from pytrends.request import TrendReq
from pathlib import Path
import pandas as pd
import math
import time

cfips = pd.read_csv(r'C:\Users\sbsla\OneDrive\Desktop\Stat_222_Proj\DATA\cfips_to_county_name.csv')

#Keywords to scrape
county_level_kws = ['tax', 'loan', 'business', 'rent', 'college', 'apartments', 'office', 'insurance'] #e.g. "XX county tax"
state_level_kws = ['tax', 'loan', 'business', 'rent', 'college', 'apartments', 'office', 'insurance', 'Business plan templates', 'Small business ideas', 'How to start a small business', 'office space for rent', 'website hosting service', 'freelancing', 'side hustle', 'ecommerce'] #e.g. "tax"

#Connect to google
pytrends = TrendReq(hl='en-US', tz=360)


In [37]:
#Pulls state level keywords for each state

states_output = dict()
for state in set(cfips['state']):
    num = math.ceil(len(state_level_kws)/5)
    df = pd.DataFrame()
    for i in range(0,num):
        keywords = state_level_kws[i*5:min(len(state_level_kws),i*5+5)]
        pytrends.build_payload(keywords, timeframe='today 5-y', geo=f'US-{state}')
        if i == 0:
            df = pytrends.interest_over_time()
            df.drop(columns=['isPartial'], inplace=True)
        else:
            result = pytrends.interest_over_time()
            result.drop(columns=['isPartial'], inplace=True)
            df = df.merge(result, how = 'left', on = 'date')
        time.sleep(6)
    states_output[state] = df

C:\Users\sbsla\AppData\Local\Temp\ipykernel_10080\1508961502.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'isPartial_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(result, how = 'left', on = 'date')
C:\Users\sbsla\AppData\Local\Temp\ipykernel_10080\1508961502.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'isPartial_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(result, how = 'left', on = 'date')
C:\Users\sbsla\AppData\Local\Temp\ipykernel_10080\1508961502.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'isPartial_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(result, how = 'left', on = 'date')
C:\Users\sbsla\AppData\Local\Temp\ipykernel_10080\1508961502.py:12: FutureWarning: Passing 'suffixes' which cause duplicate columns {'isPartial_x'} in the result is depreca

In [42]:
#stupid helper function I wrote to get the dates column

def get_dates():
    pytrends.build_payload(['dog'], timeframe='today 5-y', geo=f'US')
    df = pytrends.interest_over_time()
    df.drop(columns = ['dog'], inplace = True)
    return df

In [55]:
#Pulls county level keywords for each county

dates = get_dates() #is the seed for successive merging
counties_output = dict()
for i in range(0,len(cfips)):
    state = cfips['state'][i]
    county_name = cfips['county_name'][i]
    num = math.ceil(len(county_level_kws)/5)
    df = pd.DataFrame()
    output = dates.copy()
    for i in range(0,num):
        keywords = county_level_kws[i*5:min(len(state_level_kws),i*5+5)]
        keywords = [county_name + ' ' + keyword for keyword in keywords]
        pytrends.build_payload(keywords, timeframe='today 5-y', geo=f'US-{state}')
        df = pytrends.interest_over_time()
        if len(df) != 0:
            df.drop(columns=['isPartial'], inplace=True)
            output = output.merge(df, how = 'left', on = 'date')
        time.sleep(1)
    counties_output[f'{county_name} {state}'] = output

KeyboardInterrupt: 

In [57]:
counties_output.keys()

dict_keys(['Autauga County AL', 'Baldwin County AL', 'Barbour County AL', 'Bibb County AL', 'Blount County AL', 'Bullock County AL', 'Butler County AL', 'Calhoun County AL', 'Chambers County AL'])